In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Load dataset
dataset = load_dataset("wwbrannon/twinviews-13k")

# Convert dataset to Pandas
import pandas as pd
df = pd.DataFrame(dataset['train'])

# Convert statements into a regression dataset
df_left = df[['left']].rename(columns={'left': 'text'})
df_left['bias_score'] = -1  # Left = -1

df_right = df[['right']].rename(columns={'right': 'text'})
df_right['bias_score'] = 1  # Right = +1

df_combined = pd.concat([df_left, df_right]).sample(frac=1).reset_index(drop=True)

# Tokenization
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(df_combined['text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
labels = torch.tensor(df_combined['bias_score'].values, dtype=torch.float32)